In [2]:
%load_ext autoreload
%autoreload 2


In [3]:
# ============================================================
# ✅ FIX PATH FOR NOTEBOOK INSIDE /notebooks
# ============================================================
import sys, os
from pathlib import Path

# Xác định thư mục cha của notebooks/
project_root = Path(os.getcwd()).parent  # => C:\Users\User\RR_model
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))  # ⚠️ thêm project_root, KHÔNG thêm src

print("📁 Project root added:", project_root)

# Kiểm tra lại
import importlib.util
print("🔍 src module found:", importlib.util.find_spec("src"))

📁 Project root added: C:\Users\MAFC4709\Python_work\RR_model
🔍 src module found: ModuleSpec(name='src', loader=<_frozen_importlib_external.SourceFileLoader object at 0x000001FC305930D0>, origin='C:\\Users\\MAFC4709\\Python_work\\RR_model\\src\\__init__.py', submodule_search_locations=['C:\\Users\\MAFC4709\\Python_work\\RR_model\\src'])


In [3]:
# ===========================================================
# 📘 Roll Rate Demo v7 - Sử dụng dữ liệu thật từ Oracle hoặc SQL file
# ===========================================================

from pathlib import Path
import sys
import pandas as pd
import numpy as np

# --- 1️⃣ Setup đường dẫn và import các module ---
root = Path(".").resolve()
sys.path.append(str(root / "src"))

from src.data_loader import load_data
from src.config import CFG, OUT_ROOT
from src.rollrate.transition import compute_transition_from_pairs, compute_transition_by_mob

from src.rollrate.export_excel import export_transition_excel
from src.rollrate.backtest import (
    compute_actual_matrix,
    rollforward_validation,
    plot_matrix_diff,
    plot_distribution_compare
)

print("✅ Import modules thành công.")
print("Cấu hình CFG:", CFG)


# --- 2️⃣ Load dữ liệu thực tế ---
sql_path = r"C:/Users/MAFC4709/Python_work/RR_model/sql/LGD.sql"

df = load_data(sql_path)

df['DISBURSAL_DATE'] = df['DISBURSAL_DATE'].apply(lambda x: x.replace(day=1))

#print(f"📦 Đã load {len(df):,} dòng dữ liệu từ {sql_path}")

# Kiểm tra các cột quan trọng
required_cols = [CFG["loan"], CFG["mob"], CFG["state"], CFG["ead"], CFG["cutoff"]]
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"❌ Thiếu các cột cần thiết trong data: {missing}")

print("✅ Dữ liệu có đủ các cột cần thiết:", required_cols)

# Nếu chưa có PRODUCT_TYPE thì thêm mặc định
if "PRODUCT_TYPE" not in df.columns:
    df["PRODUCT_TYPE"] = "A"
    print("⚠️ Không có PRODUCT_TYPE trong data, gán mặc định = 'A'")

# Convert cutoff về datetime
df[CFG["cutoff"]] = pd.to_datetime(df[CFG["cutoff"]])





✅ Import modules thành công.
Cấu hình CFG: {'loan': 'AGREEMENT_ID', 'mob': 'MOB', 'state': 'STATE_MODEL', 'orig_date': 'DISBURSAL_DATE', 'ead': 'PRINCIPLE_OUTSTANDING', 'disb': 'DISBURSAL_AMOUNT', 'cutoff': 'CUTOFF_DATE'}
📦 Loading Parquet from: C:\Users\MAFC4709\Python_work\RR_model\data\parquet
✅ Loaded 25,998,763 rows via pyarrow.dataset from C:\Users\MAFC4709\Python_work\RR_model\data\parquet
✅ Dữ liệu có đủ các cột cần thiết: ['AGREEMENT_ID', 'MOB', 'STATE_MODEL', 'PRINCIPLE_OUTSTANDING', 'CUTOFF_DATE']


In [7]:
import pandas as pd
import numpy as np

from src.rollrate.transition import compute_transition_by_mob
from src.rollrate.forecast import forecast_all_vintages
from src.rollrate.forecast_plan import forecast_sale_plan_by_mob

from src.rollrate.lifecycle import (
    get_actual_all_vintages_amount,
    combine_all_lifecycle_amount,
    lifecycle_to_long_df_amount,
    add_del_metrics,
    tag_forecast_rows_amount,
)

from src.rollrate.calibration import (
    compute_k_per_product_anchor,
    #apply_k_to_forecast,
    apply_k_to_sale_plan,
)

from src.rollrate.seasonality import (
    build_seasonality,
    apply_seasonality_to_lifecycle,
    apply_seasonality_to_sale_plan,
)

from src.config import BUCKETS_CANON, CFG


⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=500-)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=550-)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=600-)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=650+)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=650-)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=OTHERS)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=SALPIL, score=A)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 

In [9]:
forecast_results = forecast_all_vintages(
    df_raw=df,
    matrices_by_mob=matrices_by_mob,
    max_mob=29,
    enable_macro=False,
)
print("✔ Forecast disbursed portfolio done")


✔ Forecast disbursed portfolio done


In [10]:
actual_results = get_actual_all_vintages_amount(df)

lifecycle = combine_all_lifecycle_amount(actual_results, forecast_results)

df_lifecycle = lifecycle_to_long_df_amount(lifecycle)
print(df_lifecycle.head())


  PRODUCT_TYPE RISK_SCORE VINTAGE_DATE  MOB        DPD0      DPD1+     DPD30+  \
0       CDLPIL       500-   2023-01-01    0  493.374711   3.613740   0.000000   
1       CDLPIL       500-   2023-01-01    1  418.023702  21.318179   0.000000   
2       CDLPIL       500-   2023-01-01    2  308.596853  53.241728  18.335200   
3       CDLPIL       500-   2023-01-01    3  265.708119   5.770333  28.830319   
4       CDLPIL       500-   2023-01-01    4  180.169614  28.523850   5.770333   

      DPD60+     DPD90+  DPD120+  DPD180+  PREPAY  WRITEOFF  SOLDOUT  
0   0.000000   0.000000      0.0      0.0     0.0       0.0      0.0  
1   0.000000   0.000000      0.0      0.0     0.0       0.0      0.0  
2   0.000000   0.000000      0.0      0.0     0.0       0.0      0.0  
3  13.005200   0.000000      0.0      0.0     0.0       0.0      0.0  
4  20.863773  17.520001      0.0      0.0     0.0       0.0      0.0  


In [11]:
df_lifecycle = tag_forecast_rows_amount(df_lifecycle, df)
print("✔ Tagged forecast rows")


✔ Tagged forecast rows


In [12]:
df_lifecycle = add_del_metrics(df_lifecycle, df)
print(df_lifecycle.head())


  PRODUCT_TYPE RISK_SCORE VINTAGE_DATE  MOB        DPD0      DPD1+     DPD30+  \
0       CDLPIL       500-   2023-01-01    0  493.374711   3.613740   0.000000   
1       CDLPIL       500-   2023-01-01    1  418.023702  21.318179   0.000000   
2       CDLPIL       500-   2023-01-01    2  308.596853  53.241728  18.335200   
3       CDLPIL       500-   2023-01-01    3  265.708119   5.770333  28.830319   
4       CDLPIL       500-   2023-01-01    4  180.169614  28.523850   5.770333   

      DPD60+     DPD90+  DPD120+  ...  SOLDOUT  ACTUAL_MAX_MOB  IS_FORECAST  \
0   0.000000   0.000000      0.0  ...      0.0              24            0   
1   0.000000   0.000000      0.0  ...      0.0              24            0   
2   0.000000   0.000000      0.0  ...      0.0              24            0   
3  13.005200   0.000000      0.0  ...      0.0              24            0   
4  20.863773  17.520001      0.0  ...      0.0              24            0   

   DISB_TOTAL  DEL30_AMT  DEL60_AMT  D

In [15]:
sale_plan_df = pd.read_excel("C:/Users/MAFC4709/Python_work/RR_model/sale_plan_5yF2.xlsx", sheet_name="SALE_PLAN")
sale_plan_df["VINTAGE_DATE"] = pd.to_datetime(sale_plan_df["VINTAGE_DATE"])

# Chuẩn hóa SCORE
def normalize_score(x):
    if pd.isna(x):
        return "ALL"
    s = str(x).strip().upper()
    return "ALL" if s in ["", "NA", "NAN", "NULL"] else s

sale_plan_df["RISK_SCORE"] = sale_plan_df["RISK_SCORE"].apply(normalize_score)

sale_plan_df.head()


,PRODUCT_TYPE,VINTAGE_DATE,RISK_SCORE,EAD_PLAN
0,CDLPIL,2025-11-01,500-,212.275694
1,CDLPIL,2025-11-01,550-,139.926121
2,CDLPIL,2025-11-01,600-,117.960122
3,CDLPIL,2025-11-01,650-,90.334822
4,CDLPIL,2025-11-01,650+,103.520520


In [16]:
states = list(dict.fromkeys(list(BUCKETS_CANON) + ["WRITEOFF", "PREPAY"]))

sale_plan_fc = forecast_sale_plan_by_mob(
    sale_plan_df=sale_plan_df,
    matrices_by_mob=matrices_by_mob,
    parent_fallback=parent_fallback,
    mob_target=24,
    start_state=BUCKETS_CANON[0],
    states=states,
)

print("✔ Forecast SALE PLAN done")
sale_plan_fc.head()


✔ Forecast SALE PLAN done


,PRODUCT_TYPE,RISK_SCORE,VINTAGE_DATE,MOB,DPD0,DPD1+,DPD30+,DPD60+,DPD90+,DPD120+,DPD180+,PREPAY,WRITEOFF,SOLDOUT
0,CDLPIL,500-,2025-11-01,0,212.275694,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0
1,CDLPIL,500-,2025-11-01,1,202.659381,6.623849,0.000000,0.000000,0.000000,0.0,0.0,2.992465,0.000000,0.0
2,CDLPIL,500-,2025-11-01,2,191.840401,11.035736,4.580148,0.024828,0.000000,0.0,0.0,4.794581,0.000000,0.0
3,CDLPIL,500-,2025-11-01,3,180.340670,13.034343,8.175745,4.098647,0.093475,0.0,0.0,6.532814,0.000000,0.0
4,CDLPIL,500-,2025-11-01,4,160.759822,22.010870,9.857140,7.071895,4.224060,0.0,0.0,8.350973,0.000935,0.0


In [20]:
from src.rollrate.calibration import compute_k_per_product_anchor

k_dict = compute_k_per_product_anchor(
    df_lifecycle=df_lifecycle,        # actual full
    forecast_long_df=df_forecast_long # forecast only
)

print(k_dict)


NameError: name 'df_forecast_long' is not defined

In [17]:
seasonality = build_seasonality(df_lifecycle)
print("✔ Seasonality ready")


✔ Seasonality ready


In [18]:
df_lifecycle = apply_seasonality_to_lifecycle(df_lifecycle, seasonality)
sale_plan_fc = apply_seasonality_to_sale_plan(sale_plan_fc, seasonality)

print("✔ Applied seasonality")


✔ Applied seasonality


In [19]:
# Gán mapping anchor MOB
anchor_map = {
    "CDLPIL": 12,
    "TWLPIL": 12,
    "SPLPIL": 12,
    # default cho các product khác
}

k_dict = compute_k_per_product_anchor(
    df_raw=df,
    forecast_results=forecast_results,
    anchor_map=anchor_map,
)

print(k_dict)


TypeError: compute_k_per_product_anchor() got an unexpected keyword argument 'df_raw'

In [ ]:
df_lifecycle = apply_k_to_forecast(df_lifecycle, k_dict)
sale_plan_fc = apply_k_to_sale_plan(sale_plan_fc, k_dict)


In [ ]:
from src.rollrate.export import export_lifecycle_all_products_one_file

# Xây actual_info cho export
actual_info = df_lifecycle.groupby(["PRODUCT_TYPE","VINTAGE_DATE"])["ACTUAL_MAX_MOB"].max().to_dict()

export_lifecycle_all_products_one_file(
    df_del_prod=df_lifecycle,
    actual_info=actual_info,
    filename="Lifecycle_Report.xlsx"
)


######

In [21]:
df_actual_lc = lifecycle_to_long_df_amount(
    combine_all_lifecycle_amount(
        get_actual_all_vintages_amount(df),
        forecast_results
    )
)

df_actual_lc = tag_forecast_rows_amount(df_actual_lc, df)
df_actual_lc = add_del_metrics(df_actual_lc, df)

print("✔ Actual lifecycle long-format:", df_actual_lc.shape)
df_actual_lc.head()


✔ Actual lifecycle long-format: (19170, 23)


,PRODUCT_TYPE,RISK_SCORE,VINTAGE_DATE,MOB,DPD0,DPD1+,DPD30+,DPD60+,DPD90+,DPD120+,...,SOLDOUT,ACTUAL_MAX_MOB,IS_FORECAST,DISB_TOTAL,DEL30_AMT,DEL60_AMT,DEL90_AMT,DEL30_PCT,DEL60_PCT,DEL90_PCT
0,CDLPIL,500-,2023-01-01,0,493.374711,3.613740,0.000000,0.000000,0.000000,0.0,...,0.0,24,0,498.44461,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,CDLPIL,500-,2023-01-01,1,418.023702,21.318179,0.000000,0.000000,0.000000,0.0,...,0.0,24,0,498.44461,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,CDLPIL,500-,2023-01-01,2,308.596853,53.241728,18.335200,0.000000,0.000000,0.0,...,0.0,24,0,498.44461,18.335200,0.000000,0.000000,0.036785,0.000000,0.000000
3,CDLPIL,500-,2023-01-01,3,265.708119,5.770333,28.830319,13.005200,0.000000,0.0,...,0.0,24,0,498.44461,41.835519,13.005200,0.000000,0.083932,0.026092,0.000000
4,CDLPIL,500-,2023-01-01,4,180.169614,28.523850,5.770333,20.863773,17.520001,0.0,...,0.0,24,0,498.44461,44.154107,38.383774,17.520001,0.088584,0.077007,0.035149


In [22]:
# =============================
# 6) FORECAST SALE PLAN
# =============================

states = list(BUCKETS_CANON) + ["WRITEOFF"]  # STATE_SPACE tạm

sale_plan_fc = forecast_sale_plan_by_mob(
    sale_plan_df=sale_plan_df,
    matrices_by_mob=matrices_by_mob,
    parent_fallback=parent_fallback,
    mob_target=24,
    start_state=BUCKETS_CANON[0],
    states=states
)

sale_plan_fc = add_del_metrics(sale_plan_fc, sale_plan_df)

print("✔ Forecast SALE PLAN xong:", sale_plan_fc.shape)
sale_plan_fc.head()


KeyError: 'DISBURSAL_DATE'

In [ ]:
# =============================
# 7) CALIBRATION k PER PRODUCT
# =============================

# Chỉ lấy phần forecast của actual
df_forecast_long = df_actual_lc[df_actual_lc["IS_FORECAST"] == 1]

k_dict = compute_k_per_product_anchor(
    df_lifecycle=df_actual_lc,
    forecast_long_df=df_forecast_long
)

print("✔ k per product:")
k_dict


In [ ]:
df_actual_calib = apply_k_to_lifecycle(df_actual_lc, k_dict)
sale_plan_calib = apply_k_to_sale_plan(sale_plan_fc, k_dict)

print("✔ Calibration applied")


In [ ]:
# =============================
# 8) SEASONALITY
# =============================
seasonality = build_seasonality(df_actual_calib)

df_actual_season = apply_seasonality_to_lifecycle(
    df_actual_calib,
    seasonality
)

sale_plan_season = apply_seasonality_to_sale_plan(
    sale_plan_calib,
    seasonality
)

print("✔ Seasonality applied")


In [ ]:
# =============================
# 9) COMBINE ACTUAL + SALE PLAN
# =============================
df_combined = pd.concat([df_actual_season, sale_plan_season], ignore_index=True)
df_combined = df_combined.sort_values(
    ["PRODUCT_TYPE", "RISK_SCORE", "VINTAGE_DATE", "MOB"]
)

print("✔ Combined lifecycle:", df_combined.shape)
df_combined.head()


In [ ]:
df_prod = aggregate_to_product(df_combined)
print("✔ Aggregated to PRODUCT:", df_prod.shape)


In [23]:
df_port = aggregate_products_to_portfolio(df_prod)
print("✔ Aggregated to PORTFOLIO:", df_port.shape)


NameError: name 'aggregate_products_to_portfolio' is not defined

In [24]:
actual_info = (
    df_actual_season[df_actual_season["IS_FORECAST"] == 0]
    .groupby(["PRODUCT_TYPE", "VINTAGE_DATE"])["MOB"]
    .max()
    .to_dict()
)


NameError: name 'df_actual_season' is not defined

In [28]:
# ============================================================
# ONE-CELL FULL PIPELINE FOR LIFECYCLE + SALE PLAN + K-CALIB + SEASON + EXPORT
# ============================================================

print("🔥 START FULL PIPELINE…")

# ============================================================
# 0) Autoreload (optional)
# ============================================================
%load_ext autoreload
%autoreload 2


# ============================================================
# 1) IMPORT
# ============================================================
import pandas as pd
import numpy as np

from src.config import CFG, BUCKETS_CANON

# Transition
from src.rollrate.transition import (
    make_pairs,
    compute_transition_by_mob
)

# Forecast engine
from src.rollrate.forecast import forecast_all_vintages
from src.rollrate.forecast_plan import forecast_sale_plan_by_mob

# Lifecycle & aggregation & export
from src.rollrate.lifecycle import (
    get_actual_all_vintages_amount,
    combine_all_lifecycle_amount,
    lifecycle_to_long_df_amount,
    tag_forecast_rows_amount,
    add_del_metrics,
    aggregate_to_product,
    aggregate_products_to_portfolio,
    export_lifecycle_all_products_one_file,
)

# Calibration
from src.rollrate.calibration import (
    compute_k_per_product_anchor,
    apply_k_to_lifecycle,
    apply_k_to_sale_plan,
)

# Seasonality
from src.rollrate.seasonality import (
    build_seasonality,
    apply_seasonality_to_lifecycle,
    apply_seasonality_to_sale_plan,
)


# ============================================================
# 2) LOAD RAW DATA
# ============================================================
# df = pd.read_parquet("C:/Users/User/Python_work/RR_model/data/raw.parquet")
# df["DISBURSAL_DATE"] = pd.to_datetime(df["DISBURSAL_DATE"])
# df["MONTH"] = df["DISBURSAL_DATE"].dt.to_period("M")
# print("✔ RAW loaded:", df.shape)


# ============================================================
# 3) LOAD SALE PLAN
# ============================================================
# sale_plan_df = pd.read_excel(
#     "C:/Users/User/Python_work/RR_model/sale_plan.xlsx",
#     sheet_name="SALE_PLAN"
# )
# sale_plan_df["VINTAGE_DATE"] = pd.to_datetime(sale_plan_df["VINTAGE_DATE"])

def normalize_score(x):
    if pd.isna(x): return "ALL"
    s = str(x).strip().upper()
    if s in ["", "NA", "NAN", "NONE", "NULL"]: return "ALL"
    return s

sale_plan_df["RISK_SCORE"] = sale_plan_df["RISK_SCORE"].apply(normalize_score)
print("✔ SALE PLAN loaded:", sale_plan_df.shape)


# ============================================================
# 4) TRANSITION MATRICES
# ============================================================
matrices_by_mob, parent_fallback = compute_transition_by_mob(df)
print("✔ Transition matrices ready")



# ============================================================
# 5) FORECAST ACTUAL LOANS (MARKOV)
# ============================================================
forecast_results = forecast_all_vintages(
    df_raw=df,
    matrices_by_mob=matrices_by_mob,
    max_mob=29
)
print("✔ Actual forecast done")

# Convert → Long lifecycle (Actual + Forecast)
df_actual_lc = lifecycle_to_long_df_amount(
    combine_all_lifecycle_amount(
        get_actual_all_vintages_amount(df),
        forecast_results
    )
)

df_actual_lc = tag_forecast_rows_amount(df_actual_lc, df)
df_actual_lc = add_del_metrics(df_actual_lc, df)
print("✔ Actual lifecycle built:", df_actual_lc.shape)


# ============================================================



🔥 START FULL PIPELINE…
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
✔ SALE PLAN loaded: (1240, 4)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=500-)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=550-)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=600-)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=650+)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=650-)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=OTHERS)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'D

KeyError: 'DISBURSAL_DATE'

In [29]:
# 6) FORECAST SALE PLAN
# ============================================================


states = list(dict.fromkeys(list(BUCKETS_CANON) + ["WRITEOFF", "PREPAY"]))

sale_plan_fc = forecast_sale_plan_by_mob(
    sale_plan_df=sale_plan_df,
    matrices_by_mob=matrices_by_mob,
    parent_fallback=parent_fallback,
    mob_target=24,
    start_state=BUCKETS_CANON[0],
    states=states,
)

print("✔ Forecast SALE PLAN done")
sale_plan_fc.head()


# ============================================================
# 7) CALIBRATION — k per product (Anchor MOB)
# ============================================================
df_forecast_only = df_actual_lc[df_actual_lc["IS_FORECAST"] == 1]

k_dict = compute_k_per_product_anchor(
    df_lifecycle=df_actual_lc,
    forecast_long_df=df_forecast_only
)

print("✔ k per product =", k_dict)

df_actual_calib = apply_k_to_lifecycle(df_actual_lc, k_dict)
sale_plan_calib = apply_k_to_sale_plan(sale_plan_fc, k_dict)
print("✔ Calibration applied")


# ============================================================
# 8) SEASONALITY
# ============================================================
seasonality = build_seasonality(df_actual_calib)
print("✔ Seasonality factors built")

df_actual_season = apply_seasonality_to_lifecycle(df_actual_calib, seasonality)
sale_plan_season = apply_seasonality_to_sale_plan(sale_plan_calib, seasonality)
print("✔ Seasonality applied")


# ============================================================
# 9) COMBINE ACTUAL + SALE PLAN
# ============================================================
df_combined = pd.concat(
    [df_actual_season, sale_plan_season],
    ignore_index=True
).sort_values(
    ["PRODUCT_TYPE", "RISK_SCORE", "VINTAGE_DATE", "MOB"]
)

print("✔ Combined lifecycle:", df_combined.shape)


# ============================================================
# 10) PRODUCT LEVEL AGGREGATION
# ============================================================
df_prod = aggregate_to_product(df_combined)
print("✔ Product-level aggregated:", df_prod.shape)


# ============================================================
# 11) PORTFOLIO LEVEL
# ============================================================
df_port = aggregate_products_to_portfolio(df_prod)
print("✔ Portfolio aggregated:", df_port.shape)


# ============================================================
# 12) ACTUAL INFO (for coloring in Excel)
# ============================================================
df_actual_season["VINTAGE_DATE"] = pd.to_datetime(df_actual_season["VINTAGE_DATE"])

actual_info = (
    df_actual_season[df_actual_season["IS_FORECAST"] == 0]
    .groupby(["PRODUCT_TYPE", "VINTAGE_DATE"])["MOB"]
    .max()
    .to_dict()
)

print("✔ actual_info size:", len(actual_info))


# ============================================================
# 13) EXPORT REPORT
# ============================================================
df_export = pd.concat([df_prod, df_port], ignore_index=True)

export_lifecycle_all_products_one_file(
    df_del_prod=df_export,
    actual_info=actual_info,
    filename="Lifecycle_Report_Final.xlsx"
)

print("🎉 EXPORT DONE → Lifecycle_Report_Final.xlsx")
print("🔥 FULL PIPELINE COMPLETED SUCCESSFULLY!")

✔ Forecast SALE PLAN done
✔ k per product = {'CDLPIL': 1.0, 'SALPIL': 1.0, 'SPLPIL': 1.0, 'TOPUP': 1.0, 'TWLPIL': 1.0, 'XSELL': 1.0}
✔ Calibration applied
✔ Seasonality factors built
✔ Seasonality applied
✔ Combined lifecycle: (50170, 26)


C:\Users\MAFC4709\Python_work\RR_model\src\rollrate\lifecycle.py:371: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


✔ Product-level aggregated: (14730, 7)


C:\Users\MAFC4709\Python_work\RR_model\src\rollrate\lifecycle.py:437: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(


✔ Portfolio aggregated: (2540, 7)
✔ actual_info size: 181
✔ Export lifecycle multi-product thành công → Lifecycle_Report_Final.xlsx
🎉 EXPORT DONE → Lifecycle_Report_Final.xlsx
🔥 FULL PIPELINE COMPLETED SUCCESSFULLY!


### 2222222222222222

In [31]:
# ============================================================
# ONE-CELL FULL PIPELINE (ĐÃ FIX LỖI add_del_metrics CHO SALE PLAN)
# ============================================================

print("🔥 START FULL PIPELINE…")

%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

from src.config import CFG, BUCKETS_CANON

# Transition
from src.rollrate.transition import make_pairs, compute_transition_by_mob

# Forecast engine
from src.rollrate.forecast import forecast_all_vintages
from src.rollrate.forecast_plan import forecast_sale_plan_by_mob

# Lifecycle, agg, export
from src.rollrate.lifecycle import (
    get_actual_all_vintages_amount,
    combine_all_lifecycle_amount,
    lifecycle_to_long_df_amount,
    tag_forecast_rows_amount,
    add_del_metrics,                # ✅ chỉ dùng cho df_raw-based lifecycle
    aggregate_to_product,
    aggregate_products_to_portfolio,
    export_lifecycle_all_products_one_file,
)

# Calibration
from src.rollrate.calibration import (
    compute_k_per_product_anchor,
    apply_k_to_lifecycle,
    apply_k_to_sale_plan,
)

# Seasonality
from src.rollrate.seasonality import (
    build_seasonality,
    apply_seasonality_to_lifecycle,
    apply_seasonality_to_sale_plan,
)

# ------------------------------------------------------------
# Helper: DEL metrics cho SALE PLAN (KHÔNG dùng loan-level DISB)
# ------------------------------------------------------------
from src.rollrate.lifecycle import BUCKETS_30P, BUCKETS_60P, BUCKETS_90P

def add_del_metrics_for_sale_plan(df_plan_fc: pd.DataFrame) -> pd.DataFrame:
    """
    Tính DEL30/60/90 AMT + PCT cho SALE PLAN forecast (sale_plan_fc).

    Input df_plan_fc: output của forecast_sale_plan_by_mob, gồm:
        - PRODUCT_TYPE, RISK_SCORE, VINTAGE_DATE, MOB
        - các bucket DPD0, DPD30+, DPD60+, DPD90+, DPD120+, DPD180+, WRITEOFF, ...

    Logic:
        - DISB_TOTAL của cohort (Product × Score × Vintage)
          = tổng EAD tại MOB0, state DPD0
        - DELxx_AMT = sum bucket tương ứng
        - DELxx_PCT = DELxx_AMT / DISB_TOTAL
        - IS_FORECAST = 1
        - SOURCE = "SALE_PLAN"
    """
    df = df_plan_fc.copy()

    key_cols = ["PRODUCT_TYPE", "RISK_SCORE", "VINTAGE_DATE"]

    # 1️⃣ DISB_TOTAL ~ tổng EAD tại MOB=0 & DPD0 (coi như tổng disb plan)
    disb_plan = (
        df[df["MOB"] == 0]
        .groupby(key_cols)["DPD0"]
        .sum()
        .rename("DISB_TOTAL")
        .reset_index()
    )

    df = df.merge(disb_plan, on=key_cols, how="left")

    # 2️⃣ DEL AMOUNT
    df["DEL30_AMT"] = df[BUCKETS_30P].sum(axis=1)
    df["DEL60_AMT"] = df[BUCKETS_60P].sum(axis=1)
    df["DEL90_AMT"] = df[BUCKETS_90P].sum(axis=1)

    # 3️⃣ DEL PCT trên DISB_TOTAL (giống lifecycle)
    df["DEL30_PCT"] = df["DEL30_AMT"] / df["DISB_TOTAL"]
    df["DEL60_PCT"] = df["DEL60_AMT"] / df["DISB_TOTAL"]
    df["DEL90_PCT"] = df["DEL90_AMT"] / df["DISB_TOTAL"]

    # 4️⃣ Flag
    df["IS_FORECAST"] = 1
    df["SOURCE"] = "SALE_PLAN"

    return df



# ============================================================
# 1) LOAD RAW DATA
# ============================================================


# ============================================================
# 2) LOAD SALE PLAN
# ============================================================

sale_plan_df["VINTAGE_DATE"] = pd.to_datetime(sale_plan_df["VINTAGE_DATE"])

def normalize_score(x):
    if pd.isna(x): return "ALL"
    s = str(x).strip().upper()
    if s in ["", "NA", "NAN", "NONE", "NULL"]:
        return "ALL"
    return s

sale_plan_df["RISK_SCORE"] = sale_plan_df["RISK_SCORE"].apply(normalize_score)
print("✔ SALE PLAN loaded:", sale_plan_df.shape)

# ============================================================
# 3) TRANSITION MATRICES
# ============================================================

matrices_by_mob, parent_fallback = compute_transition_by_mob(df)
print("✔ Transition matrices built")

# ============================================================
# 4) FORECAST ACTUAL LOANS
# ============================================================
forecast_results = forecast_all_vintages(
    df_raw=df,
    matrices_by_mob=matrices_by_mob,
    max_mob=29
)
print("✔ Actual forecast done")

df_actual_lc = lifecycle_to_long_df_amount(
    combine_all_lifecycle_amount(
        get_actual_all_vintages_amount(df),
        forecast_results
    )
)

df_actual_lc = tag_forecast_rows_amount(df_actual_lc, df)
df_actual_lc = add_del_metrics(df_actual_lc, df)   # ✅ dùng loan-level df
print("✔ Actual lifecycle:", df_actual_lc.shape)

# ============================================================
# 5) FORECAST SALE PLAN
# ============================================================


from src.rollrate.lifecycle import BUCKETS_30P, BUCKETS_60P, BUCKETS_90P

sale_plan_fc = forecast_sale_plan_by_mob(
    sale_plan_df=sale_plan_df,
    matrices_by_mob=matrices_by_mob,
    parent_fallback=parent_fallback,
    mob_target=24,
    start_state=BUCKETS_CANON[0],
    states=states
)

sale_plan_fc = add_del_metrics_for_sale_plan(sale_plan_fc)
print("✔ Sale plan forecast + DEL metrics:", sale_plan_fc.shape)

print("✔ Sale plan forecast (with DEL metrics):", sale_plan_fc.shape)

# ============================================================
# 6) CALIBRATION k PER PRODUCT (anchor MOB)
# ============================================================
df_forecast_only = df_actual_lc[df_actual_lc["IS_FORECAST"] == 1]

k_dict = compute_k_per_product_anchor(
    df_lifecycle=df_actual_lc,
    forecast_long_df=df_forecast_only
)
print("✔ k per product:", k_dict)

df_actual_calib  = apply_k_to_lifecycle(df_actual_lc, k_dict)
sale_plan_calib  = apply_k_to_sale_plan(sale_plan_fc, k_dict)
print("✔ Calibration applied")

# ============================================================
# 7) SEASONALITY
# ============================================================
seasonality = build_seasonality(df_actual_calib)
print("✔ Seasonality built")

df_actual_season = apply_seasonality_to_lifecycle(df_actual_calib, seasonality)
sale_plan_season = apply_seasonality_to_sale_plan(sale_plan_calib, seasonality)
print("✔ Seasonality applied")

# ============================================================
# 8) COMBINE ACTUAL + SALE PLAN
# ============================================================
df_combined = pd.concat([df_actual_season, sale_plan_season], ignore_index=True)
df_combined = df_combined.sort_values(
    ["PRODUCT_TYPE", "RISK_SCORE", "VINTAGE_DATE", "MOB"]
)
print("✔ Combined lifecycle:", df_combined.shape)

# ============================================================
# 9) PRODUCT & PORTFOLIO AGGREGATION
# ============================================================
df_prod = aggregate_to_product(df_combined)
print("✔ Product-level:", df_prod.shape)

df_port = aggregate_products_to_portfolio(df_prod)
print("✔ Portfolio-level:", df_port.shape)

# ============================================================
# 10) ACTUAL_INFO & EXPORT
# ============================================================
actual_info = (
    df_actual_season[df_actual_season["IS_FORECAST"] == 0]
    .groupby(["PRODUCT_TYPE", "VINTAGE_DATE"])["MOB"]
    .max()
    .to_dict()
)

df_export = pd.concat([df_prod, df_port], ignore_index=True)

export_lifecycle_all_products_one_file(
    df_del_prod=df_export,
    actual_info=actual_info,
    filename="Lifecycle_Report_Final.xlsx"
)

print("🎉 EXPORT DONE → Lifecycle_Report_Final.xlsx")
print("🔥 FULL PIPELINE COMPLETED (NO DISBURSAL_DATE ERROR).")


🔥 START FULL PIPELINE…
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
✔ SALE PLAN loaded: (1240, 4)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=500-)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=550-)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=600-)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=650+)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=650-)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=OTHERS)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'D

C:\Users\MAFC4709\Python_work\RR_model\src\rollrate\lifecycle.py:371: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


✔ Product-level: (14730, 7)


C:\Users\MAFC4709\Python_work\RR_model\src\rollrate\lifecycle.py:437: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(


✔ Portfolio-level: (2540, 7)


PermissionError: [Errno 13] Permission denied: 'Lifecycle_Report_Final.xlsx'

In [32]:
export_lifecycle_all_products_one_file(
    df_del_prod=df_export,
    actual_info=actual_info,
    filename="Lifecycle_Report_Final.xlsx"
)

✔ Export lifecycle multi-product thành công → Lifecycle_Report_Final.xlsx


In [33]:
print("✔ k per product:", k_dict)

✔ k per product: {'CDLPIL': 1.0, 'SALPIL': 1.0, 'SPLPIL': 1.0, 'TOPUP': 1.0, 'TWLPIL': 1.0, 'XSELL': 1.0}


In [37]:
from src.rollrate.calibration import (
    compute_k_per_product_anchor,
    apply_k_to_lifecycle,
    apply_k_to_sale_plan,
    get_anchor_for_product
)


In [38]:
for prod in df_lifecycle["PRODUCT_TYPE"].unique():
    H = get_anchor_for_product(prod)
    print(prod, "anchor=", H)

    print("Actual rows:", df_lifecycle[df_lifecycle["MOB"] == H].shape)
    print("Forecast rows:", df_forecast_long[df_forecast_long["MOB"] == H].shape)
    print("-----")


CDLPIL anchor= 12
Actual rows: (639, 25)


NameError: name 'df_forecast_long' is not defined

In [39]:
from src.rollrate.forecast import forecast_to_dataframe
import pandas as pd

def forecast_dict_to_long_df(forecast_results):
    """
    Convert forecast_all_vintages() → long DataFrame:
        PRODUCT_TYPE
        RISK_SCORE
        VINTAGE_DATE
        MOB
        (EAD per state)
    """
    rows = []

    for (product, score, vintage), mob_dict in forecast_results.items():
        for mob, ead_vec in mob_dict.items():

            row = {
                "PRODUCT_TYPE": product,
                "RISK_SCORE": score,
                "VINTAGE_DATE": vintage,
                "MOB": mob
            }
            row.update(ead_vec.to_dict())
            rows.append(row)

    return pd.DataFrame(rows)


In [53]:
from src.rollrate.forecast import forecast_to_dataframe
from src.rollrate.calibration import compute_k_per_product_anchor,extract_actual_del90, extract_forecast_del90
from src.rollrate.lifecycle import BUCKETS_90P


In [41]:
df_forecast_long = forecast_dict_to_long_df(forecast_results)


In [50]:
df_forecast_long

,PRODUCT_TYPE,RISK_SCORE,VINTAGE_DATE,MOB,DPD0,DPD1+,DPD30+,DPD60+,DPD90+,DPD120+,DPD180+,PREPAY,WRITEOFF,SOLDOUT
0,CDLPIL,500-,2023-01-01,24,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,81.127901,0.0
1,CDLPIL,500-,2023-01-01,25,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,81.127901,0.0
2,CDLPIL,500-,2023-01-01,26,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,81.127901,0.0
3,CDLPIL,500-,2023-01-01,27,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,81.127901,0.0
4,CDLPIL,500-,2023-01-01,28,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,81.127901,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9793,XSELL,D,2025-09-01,25,12960.646062,1404.289727,1404.570635,744.831512,2742.671920,0.0,0.0,147.622141,12909.705059,0.0
9794,XSELL,D,2025-09-01,26,12273.840800,1361.200280,1450.751230,822.741675,2676.603543,0.0,0.0,178.883358,13550.316171,0.0
9795,XSELL,D,2025-09-01,27,11624.882841,1308.619218,1449.920319,862.544953,2663.543262,0.0,0.0,208.708576,14196.117888,0.0
9796,XSELL,D,2025-09-01,28,11011.136400,1251.602885,1420.926154,871.901341,2673.478934,0.0,0.0,237.095635,14848.195709,0.0


In [42]:
for prod in df_lifecycle["PRODUCT_TYPE"].unique():
    H = get_anchor_for_product(prod)
    print(prod, "anchor=", H)

    print("Actual rows:", df_lifecycle[df_lifecycle["MOB"] == H].shape)
    print("Forecast rows:", df_forecast_long[df_forecast_long["MOB"] == H].shape)
    print("-----")


CDLPIL anchor= 12
Actual rows: (639, 25)
Forecast rows: (260, 14)
-----
SALPIL anchor= 24
Actual rows: (639, 25)
Forecast rows: (639, 14)
-----
SPLPIL anchor= 12
Actual rows: (639, 25)
Forecast rows: (260, 14)
-----
TOPUP anchor= 24
Actual rows: (639, 25)
Forecast rows: (639, 14)
-----
TWLPIL anchor= 12
Actual rows: (639, 25)
Forecast rows: (260, 14)
-----
XSELL anchor= 24
Actual rows: (639, 25)
Forecast rows: (639, 14)
-----


In [54]:
# 1. Tính lifecycle forecast
df_lifecycle = df_forecast_long  # hoặc build_full_lifecycle_amount_v2

# 2. Tính DEL metrics đầy đủ
df_lifecycle = add_del_metrics(df_lifecycle, df)

# 3. Lấy actual & forecast DEL90 theo MOB
df_actual   = extract_actual_del90(df_lifecycle)
df_forecast = extract_forecast_del90(df_lifecycle)

# 4. anchor MOB mapping
anchor_map = {
    "CDLPIL": 12,
    "TWLPIL": 12,
    "SPLPIL": 12,
    # default = 24
}

# 5. Tính k
k_dict = compute_k_per_product_anchor(
    df_actual,
    df_forecast,
    anchor_map
)

print(k_dict)


{'CDLPIL': 1.0, 'SALPIL': 1.0, 'SPLPIL': 1.0, 'TOPUP': 1.0, 'TWLPIL': 1.0, 'XSELL': 1.0}


In [58]:
from src.rollrate.calibration import (
    forecast_full_history,
    compute_k_per_product_ifrs,
    apply_k_to_lifecycle,
    apply_k_to_sale_plan,
)
df_lifecycle = add_del_metrics(df_lifecycle, df)
print(df_lifecycle.head())

# 1) Forecast full history (Markov thuần)
df_full_fc = forecast_full_history(
    df_raw=df,
    matrices_by_mob=matrices_by_mob,
    parent_fallback=parent_fallback,
    max_mob=30,
)

# 2) Anchor map per product
anchor_map = {
    "CDLPIL": 12,
    "TWLPIL": 12,
    "SPLPIL": 12,
    # các product khác dùng default_anchor_mob (ví dụ 24)
}

# 3) Tính k per product
k_dict = compute_k_per_product_ifrs(
    df_actual_lifecycle=df_actual_lc,   # đã add_del_metrics
    df_full_forecast=df_full_fc,
    anchor_map=anchor_map,
    default_anchor_mob=24,
    min_obs=4,
)
print("k_dict:", k_dict)

# 4) Áp k
df_actual_calib = apply_k_to_lifecycle(df_actual_lc, k_dict)
sale_plan_calib = apply_k_to_sale_plan(sale_plan_fc, k_dict)


  PRODUCT_TYPE RISK_SCORE VINTAGE_DATE  MOB        DPD0      DPD1+     DPD30+  \
0       CDLPIL       500-   2023-01-01    0  493.374711   3.613740   0.000000   
1       CDLPIL       500-   2023-01-01    1  418.023702  21.318179   0.000000   
2       CDLPIL       500-   2023-01-01    2  308.596853  53.241728  18.335200   
3       CDLPIL       500-   2023-01-01    3  265.708119   5.770333  28.830319   
4       CDLPIL       500-   2023-01-01    4  180.169614  28.523850   5.770333   

      DPD60+     DPD90+  DPD120+  ...  PREPAY  WRITEOFF  SOLDOUT  DISB_TOTAL  \
0   0.000000   0.000000      0.0  ...     0.0       0.0      0.0   498.44461   
1   0.000000   0.000000      0.0  ...     0.0       0.0      0.0   498.44461   
2   0.000000   0.000000      0.0  ...     0.0       0.0      0.0   498.44461   
3  13.005200   0.000000      0.0  ...     0.0       0.0      0.0   498.44461   
4  20.863773  17.520001      0.0  ...     0.0       0.0      0.0   498.44461   

   DEL30_AMT  DEL60_AMT  DEL90_A

KeyError: 'df_lifecycle chưa có DEL90_PCT — hãy chạy add_del_metrics trước khi calibration.'

In [57]:
actual_results = get_actual_all_vintages_amount(df)

lifecycle = combine_all_lifecycle_amount(actual_results, forecast_results)

df_lifecycle = lifecycle_to_long_df_amount(lifecycle)
print(df_lifecycle.head())

  PRODUCT_TYPE RISK_SCORE VINTAGE_DATE  MOB        DPD0      DPD1+     DPD30+  \
0       CDLPIL       500-   2023-01-01    0  493.374711   3.613740   0.000000   
1       CDLPIL       500-   2023-01-01    1  418.023702  21.318179   0.000000   
2       CDLPIL       500-   2023-01-01    2  308.596853  53.241728  18.335200   
3       CDLPIL       500-   2023-01-01    3  265.708119   5.770333  28.830319   
4       CDLPIL       500-   2023-01-01    4  180.169614  28.523850   5.770333   

      DPD60+     DPD90+  DPD120+  DPD180+  PREPAY  WRITEOFF  SOLDOUT  
0   0.000000   0.000000      0.0      0.0     0.0       0.0      0.0  
1   0.000000   0.000000      0.0      0.0     0.0       0.0      0.0  
2   0.000000   0.000000      0.0      0.0     0.0       0.0      0.0  
3  13.005200   0.000000      0.0      0.0     0.0       0.0      0.0  
4  20.863773  17.520001      0.0      0.0     0.0       0.0      0.0  


###### 33333333

In [60]:
# ============================================
# 0) AUTO-RELOAD MODULE (khuyến khích)
# ============================================
%load_ext autoreload
%autoreload 2

# ============================================
# 1) IMPORT
# ============================================
import pandas as pd

from src.config import CFG, BUCKETS_CANON

from src.rollrate.transition import (
    make_pairs,
    compute_transition_by_mob,
)

from src.rollrate.forecast import (
    forecast_all_vintages,
)

from src.rollrate.lifecycle import (
    get_actual_all_vintages_amount,
    combine_all_lifecycle_amount,
    lifecycle_to_long_df_amount,
    add_del_metrics,
)

from src.rollrate.calibration import (
    forecast_full_history,
    compute_k_per_product_ifrs,
    apply_k_to_lifecycle,
    apply_k_to_sale_plan,
)

from src.rollrate.forecast_plan import forecast_sale_plan_by_mob
from src.rollrate.lifecycle import BUCKETS_30P, BUCKETS_60P, BUCKETS_90P


# ============================================
# 2) LOAD RAW DATA
# ============================================
# df_raw = pd.read_parquet("C:/Users/User/Python_work/RR_model/data/raw.parquet")

# orig_col = CFG["orig_date"]
# df_raw[orig_col] = pd.to_datetime(df_raw[orig_col])

# print("✔ df_raw:", df_raw.shape)


# ============================================
# 3) TRANSITION MATRICES
# ============================================

matrices_by_mob, parent_fallback = compute_transition_by_mob(df)
print("✔ matrices_by_mob built")


# ============================================
# 4) LIFECYCLE THỰC + FORECAST (ENGINE CŨ)
#    → SAU ĐÓ THÊM DEL30/60/90
# ============================================

# 4.1 Forecast all vintages (engine cũ, dùng actual)
forecast_results = forecast_all_vintages(
    df_raw=df,
    matrices_by_mob=matrices_by_mob,
    max_mob=29,
    enable_macro=False,
)

# 4.2 Actual EAD theo state
actual_results = get_actual_all_vintages_amount(df)

# 4.3 Merge actual + forecast vào dict lifecycle
lifecycle_dict = combine_all_lifecycle_amount(actual_results, forecast_results)

# 4.4 Long format EAD theo bucket
df_lifecycle = lifecycle_to_long_df_amount(lifecycle_dict)

# 4.5 THÊM DEL METRICS → tạo DEL30/60/90_AMT & DEL30/60/90_PCT
df_lifecycle = add_del_metrics(df_lifecycle, df)

print("✔ df_lifecycle (sau add_del_metrics):", df_lifecycle.shape)
print("  Columns:", [c for c in df_lifecycle.columns if "DEL" in c or c in ["DISB_TOTAL"]][:10])


# ============================================
# 5) FORECAST FULL HISTORY (MARKOV THUẦN, KHÔNG DÙNG ACTUAL)
#    → DÙNG CHO CALIBRATION IFRS
# ============================================
df_full_fc = forecast_full_history(
    df_raw=df,
    matrices_by_mob=matrices_by_mob,
    parent_fallback=parent_fallback,
    max_mob=29,
)

print("✔ df_full_fc (full-history forecast):", df_full_fc.shape)


# ============================================
# 6) CALIBRATION — TÍNH k PER PRODUCT
# ============================================
anchor_map = {
    "CDLPIL": 12,
    "TWLPIL": 12,
    "SPLPIL": 12,
    # các product khác dùng default_anchor_mob = 24
}

k_dict = compute_k_per_product_ifrs(
    df_actual_lifecycle=df_lifecycle,   # đã có DEL90_PCT
    df_full_forecast=df_full_fc,
    anchor_map=anchor_map,
    default_anchor_mob=24,
    min_obs=4,
)

print("✔ k_dict =", k_dict)


# ============================================
# 7) ÁP k VÀO LIFECYCLE
# ============================================
df_lifecycle_calib = apply_k_to_lifecycle(df_lifecycle, k_dict)
print("✔ df_lifecycle_calib:", df_lifecycle_calib.shape)


# ============================================


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=500-)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=550-)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=600-)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=650+)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=650-)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=OTHERS)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallbac

In [64]:
# 8) (OPTIONAL) FORECAST SALE PLAN + ÁP k
# ============================================
# Nếu bạn chưa có sale_plan_df thì bỏ qua phần này,
# hoặc giữ lại làm template.

# Ví dụ (bạn sửa lại path cho đúng):
sale_plan_df =  pd.read_excel("C:/Users/MAFC4709/Python_work/RR_model/sale_plan_5yF2.xlsx", sheet_name="SALE_PLAN")

sale_plan_df["VINTAGE_DATE"] = pd.to_datetime(sale_plan_df["VINTAGE_DATE"])

def normalize_score(x):
    if pd.isna(x): return "ALL"
    s = str(x).strip().upper()
    if s in ["", "NA", "NAN", "NONE", "NULL"]:
        return "ALL"
    return s

sale_plan_df["RISK_SCORE"] = sale_plan_df["RISK_SCORE"].apply(normalize_score)

states = list(BUCKETS_CANON)

sale_plan_fc = forecast_sale_plan_by_mob(
    sale_plan_df=sale_plan_df,
    matrices_by_mob=matrices_by_mob,
    parent_fallback=parent_fallback,
    mob_target=24,
    start_state=BUCKETS_CANON[0],
    states=states,
)

# Áp k
sale_plan_fc_calib = apply_k_to_sale_plan(sale_plan_fc, k_dict)
print("✔ sale_plan_fc_calib:", sale_plan_fc_calib.shape)


✔ sale_plan_fc_calib: (31000, 15)


In [65]:
sale_plan_fc_calib

,PRODUCT_TYPE,RISK_SCORE,VINTAGE_DATE,MOB,DPD0,DPD1+,DPD30+,DPD60+,DPD90+,DPD120+,DPD180+,PREPAY,WRITEOFF,SOLDOUT,CALIB_K
0,CDLPIL,500-,2025-11-01,0,160.337478,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.755327
1,CDLPIL,500-,2025-11-01,1,153.074021,5.003169,0.000000,0.000000,0.000000,0.0,0.0,2.260288,0.000000,0.0,0.755327
2,CDLPIL,500-,2025-11-01,2,144.902158,8.335585,3.459508,0.018753,0.000000,0.0,0.0,3.621475,0.000000,0.0,0.755327
3,CDLPIL,500-,2025-11-01,3,136.216105,9.845186,6.175358,3.095817,0.070604,0.0,0.0,4.934408,0.000000,0.0,0.755327
4,CDLPIL,500-,2025-11-01,4,121.426170,16.625396,7.445360,5.341590,3.190545,0.0,0.0,6.307712,0.000706,0.0,0.755327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30995,XSELL,D,2030-12-01,20,19.112237,1.565895,1.499486,1.140291,4.545430,0.0,0.0,0.086283,10.733961,0.0,0.611847
30996,XSELL,D,2030-12-01,21,18.369535,1.717232,1.266173,1.158700,4.359049,0.0,0.0,0.092530,11.720365,0.0,0.611847
30997,XSELL,D,2030-12-01,22,17.981655,1.405417,1.343245,0.939965,4.084387,0.0,0.0,0.093983,12.834932,0.0,0.611847
30998,XSELL,D,2030-12-01,23,17.341796,1.699985,1.254888,0.793544,3.662231,0.0,0.0,0.097127,13.834012,0.0,0.611847


In [71]:
# ============================================================
# 9) GHÉP LIFECYCLE + SALE PLAN (sau calibration)
# ============================================================

# Nếu bạn có sale_plan_fc_calib
has_sale_plan = "sale_plan_fc_calib" in globals()

if has_sale_plan:
    df_plan = sale_plan_fc_calib.copy()
else:
    df_plan = pd.DataFrame(columns=df_lifecycle_calib.columns)  # dummy nếu không có sale plan

# ============================================================
# Đảm bảo df_lifecycle_calib có cột IS_FORECAST
# ============================================================
from src.rollrate.lifecycle import tag_forecast_rows_amount

if "IS_FORECAST" not in df_lifecycle_calib.columns:
    df_lifecycle_calib = tag_forecast_rows_amount(df_lifecycle_calib, df)

df_lifecycle_calib["SOURCE"] = "Historical"
df_lifecycle_calib["IS_FORECAST"] = df_lifecycle_calib["IS_FORECAST"].fillna(0).astype(int)


if has_sale_plan:
    df_plan["SOURCE"] = "SALE_PLAN"
    df_plan["IS_FORECAST"] = 1

# Ghép tất cả
df_all = pd.concat([df_lifecycle_calib, df_plan], ignore_index=True).sort_values(
    ["PRODUCT_TYPE", "RISK_SCORE", "VINTAGE_DATE", "MOB"]
)
print("✔ df_all:", df_all.shape)


# ============================================================
# 10) TÍNH LẠI DEL METRICS (trên df_all)
# ============================================================
df_all = add_del_metrics(df_all, df)
print("✔ add_del_metrics → done")


# ============================================================
# 11) AGGREGATE SCORE → PRODUCT
# ============================================================
from src.rollrate.lifecycle import aggregate_to_product

df_prod = aggregate_to_product(df_all)
print("✔ df_prod (product-level):", df_prod.shape)


# ============================================================
# 12) AGGREGATE PRODUCT → PORTFOLIO
# ============================================================
from src.rollrate.lifecycle import aggregate_products_to_portfolio

df_port = aggregate_products_to_portfolio(
    df_del_prod=df_prod,
    portfolio_name="PORTFOLIO_ALL",
)

print("✔ df_port (portfolio):", df_port.shape)


# ============================================================
# 13) GHÉP PRODUCT + PORTFOLIO
# ============================================================
df_prod_port = pd.concat([df_prod, df_port], ignore_index=True)
print("✔ df_prod_port:", df_prod_port.shape)


# ============================================================
# 14) BUILD actual_info (để biết vị trí actual/forecast)
# ============================================================
actual_info = (
    df_all[df_all["IS_FORECAST"] == 0]
    .groupby(["PRODUCT_TYPE", "VINTAGE_DATE"])["MOB"]
    .max()
    .to_dict()
)

# Thêm portfolio actual info
from src.rollrate.lifecycle import extend_actual_info_with_portfolio
actual_info = extend_actual_info_with_portfolio(actual_info, portfolio_name="PORTFOLIO_ALL")

print("✔ actual_info:", len(actual_info))


# ============================================================
# 15) EXPORT LIFECYCLE → EXCEL
# ============================================================
from src.rollrate.lifecycle import export_lifecycle_all_products_one_file

export_lifecycle_all_products_one_file(
    df_del_prod=df_prod_port,
    actual_info=actual_info,
    filename="Lifecycle_ALL_With_SalePlan.xlsx"
)

print("🎉 DONE — FILE EXCEL ĐÃ XUẤT HOÀN TẤT")


✔ df_all: (50170, 25)
✔ add_del_metrics → done


C:\Users\MAFC4709\Python_work\RR_model\src\rollrate\lifecycle.py:416: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .first()               # vì PRODUCT_DISB không đổi theo MOB


✔ df_prod (product-level): (14730, 7)


C:\Users\MAFC4709\Python_work\RR_model\src\rollrate\lifecycle.py:482: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ✔ ROUND 4 decimals


✔ df_port (portfolio): (2540, 7)
✔ df_prod_port: (17270, 7)
✔ actual_info: 214
✔ Export lifecycle multi-product thành công → Lifecycle_ALL_With_SalePlan.xlsx
🎉 DONE — FILE EXCEL ĐÃ XUẤT HOÀN TẤT


In [4]:

from pathlib import Path
import sys
import pandas as pd
import numpy as np

# --- 1️⃣ Setup đường dẫn và import các module ---
root = Path(".").resolve()
sys.path.append(str(root / "src"))

from src.data_loader import load_data
from src.config import CFG, OUT_ROOT
from src.rollrate.transition import compute_transition_from_pairs, compute_transition_by_mob

from src.rollrate.export_excel import export_transition_excel
from src.rollrate.backtest import (
    compute_actual_matrix,
    rollforward_validation,
    plot_matrix_diff,
    plot_distribution_compare
)

print("✅ Import modules thành công.")
print("Cấu hình CFG:", CFG)


# --- 2️⃣ Load dữ liệu thực tế ---
sql_path = r"C:/Users/MAFC4709/Python_work/RR_model/sql/LGD.sql"

df = load_data(sql_path)


✅ Import modules thành công.
Cấu hình CFG: {'loan': 'AGREEMENT_ID', 'mob': 'MOB', 'state': 'STATE_MODEL', 'orig_date': 'DISBURSAL_DATE', 'ead': 'PRINCIPLE_OUTSTANDING', 'disb': 'DISBURSAL_AMOUNT', 'cutoff': 'CUTOFF_DATE'}
🔗 Loading data from Oracle...
=== SQL DEBUG ===
File: C:\Users\MAFC4709\Python_work\RR_model\sql\LGD.sql
First 200 chars:
 SELECT * from LGD WHERE CUTOFF_DATE_M0 IS NOT null
Params: {}


C:\Users\MAFC4709\Python_work\RR_model\src\db.py:84: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(


In [5]:
import pandas as pd

# Giả sử DataFrame của bạn tên là df
df.to_parquet("LGD.parquet", compression="snappy", index=False)


In [6]:
df

,MIN_CUTOFF_DATE,CUTOFF_DATE_M0,PRODUCT,PRODUCT_2,AGREEMENT_ID,M0_DPD,M0_POS,M0_STAGE,M6_DPD,M6_POS,...,M12_DPD,M12_POS,M12_STAGE,M18_DPD,M18_POS,M18_STAGE,M24_DPD,M24_POS,M24_STAGE,FLAG_SOLDOUT
0,2023-06-21,2023-06-21,Others,None,996911,215,36.925000,SOLDOUT,NaN,NaN,...,NaN,NaN,None,NaN,NaN,None,NaN,NaN,None,1
1,2023-06-21,2023-06-21,Others,None,996968,262,15.917657,SOLDOUT,NaN,NaN,...,NaN,NaN,None,NaN,NaN,None,NaN,NaN,None,1
2,2023-06-21,2023-06-21,Others,None,996969,287,7.590573,SOLDOUT,NaN,NaN,...,NaN,NaN,None,NaN,NaN,None,NaN,NaN,None,1
3,2023-06-21,2023-06-21,Others,None,997089,287,13.437190,SOLDOUT,NaN,NaN,...,NaN,NaN,None,NaN,NaN,None,NaN,NaN,None,1
4,2023-06-21,2023-06-21,Others,None,997093,227,2.270887,SOLDOUT,NaN,NaN,...,NaN,NaN,None,NaN,NaN,None,NaN,NaN,None,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
944697,2023-12-31,2023-12-31,SURROGATE,NTB,3172301,107,23.234731,TB3,289.0,22.488274,...,0.0,0.000000,WRITEOFF,0.0,0.000000,WRITEOFF,NaN,NaN,None,0
944698,2023-06-21,2023-06-21,Others,None,3172306,195,26.420915,SOLDOUT,NaN,NaN,...,NaN,NaN,None,NaN,NaN,None,NaN,NaN,None,1
944699,2023-06-30,2023-06-30,Others,None,3172346,97,21.972200,TB3,NaN,NaN,...,NaN,NaN,None,NaN,NaN,None,NaN,NaN,None,1
944700,2023-12-31,2023-12-31,SURROGATE,NTB,3172439,107,4.726193,TB3,289.0,4.726193,...,473.0,4.726193,WRITEOFF,654.0,4.726193,WRITEOFF,NaN,NaN,None,1
